In [ ]:
import ROOT
import pandas
import math
#matplotlib.use('agg')
%matplotlib inline
import matplotlib #as matplot
print(matplotlib.__version__)
#print(matplotlib.path)
import matplotlib.pyplot as plt
from matplotlib import cm as cm
import numpy as np
import time

In [ ]:
# test that matplotlib imported ok
x1, y1 = [-1, 12], [1, 4]
x2, y2 = [1, 10], [3, 2]
plt.plot(x1, y1, x2, y2, marker = 'o')
plt.show()

In [ ]:
from sklearn.metrics import roc_curve, auc
import pylab
import sklearn as sk
print(sk.__version__)
from sklearn.model_selection import train_test_split

In [ ]:
import tensorflow as tf
print(tf.__version__)
######################
import keras as kr
from keras.models import Sequential
from keras.layers import InputLayer, Input
from keras.layers import Reshape, MaxPooling2D
from keras.layers import Conv2D, Dense, Flatten, Dropout
print(kr.__version__)
from keras import backend as K
import tensorflow as tf
print(tf.__version__)
config = tf.ConfigProto(intra_op_parallelism_threads=32, \
                        inter_op_parallelism_threads=32, \
                        allow_soft_placement=True, \
                        device_count = {'CPU': 32}
                       )
session = tf.Session(config=config)
K.set_session(session)

In [ ]:
################
## load the data
################
channel='2l_2tau_HH'
bdtType = "evtLevelSUM_HH_2l_2tau_res"
tauID = "dR03mvaLoose"

In [ ]:
%run ../python/data_manager.py
if channel=='2l_2tau_HH' : 
    %run ../cards/info_2l_2tau_HH.py
output = read_from(tauID)
print ("read from:", output["inputPath"])
print ("Date: ", time.asctime( time.localtime(time.time()) ))
data=load_data_2017(
    output["inputPath"], 
    output["channelInTree"], 
    trainVars(True), 
    [], 
    bdtType, 
    channel,
    output["keys"], 
    output["masses"],
) # note: I had to add channel as argument of the function
print (len(data))
print ("Date: ", time.asctime( time.localtime(time.time()) ))

In [ ]:
#### not finshed cell, do your fix for normalization
keys = ['ttHToNonbb', 'TTWJets', 'TTZJets', 'TTTo2L2Nu', 'TTToSemiLeptonic', 'THW_ctcvcp', 'THQ_ctcvcp' ] 
weights="evtWeight"

print ("Date: ", time.asctime( time.localtime(time.time()) ))
data.loc[(data["target"]==1) & (data[channels[nn]].values) , ["weight_cx"]]      *= output[0]/data.loc[(data["target"]==1)  & (data[channels[nn]].values), weights].sum()
data.loc[(data['key']=='TTWJets') & (data[channels[nn]].values) , ["weight_cx"]] *= output[1]/data.loc[(data['key']=='TTWJets') & (data[channels[nn]].values), weights].sum()
data.loc[(data['key']=='TTZJets') & (data[channels[nn]].values) , ["weight_cx"]] *= output[2]/data.loc[(data['key']=='TTZJets')  & (data[channels[nn]].values), weights].sum()
data.loc[(data['key'].isin(['TTTo2L2Nu','TTToSemiLeptonic', 'TTToHadronic'])) & (data[channels[nn]].values) , ["weight_cx"]] *= output[3]/data.loc[(data['key'].isin(['TTTo2L2Nu','TTToSemiLeptonic', 'TTToHadronic'])) & (data[channels[nn]].values), weights].sum()
data.loc[(data['key']=='THQ_ctcvcp') & (data[channels[nn]].values) , ["weight_cx"]] *= output[4]/data.loc[(data['key']=='THQ_ctcvcp') & (data[channels[nn]].values), weights].sum()
data.loc[(data['key']=='THW_ctcvcp') & (data[channels[nn]].values) , ["weight_cx"]] *= output[5]/data.loc[(data['key']=='THW_ctcvcp') & (data[channels[nn]].values), weights].sum()
# add 
print ("Date: ", time.asctime( time.localtime(time.time()) ))